<a href="https://colab.research.google.com/github/lindaperez/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install findspark

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
--2022-05-28 22:47:12--  http://www.apache.org/dist

In [ ]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-28 22:47:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.56MB/s    in 0.2s    

2022-05-28 22:47:34 (5.56 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)



In [ ]:
!ls

postgresql-42.2.16.jar	  spark-3.0.3-bin-hadoop2.7
postgresql-42.2.16.jar.1  spark-3.0.3-bin-hadoop2.7.tgz
sample_data		  spark-3.0.3-bin-hadoop2.7.tgz.1


In [ ]:
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [ ]:
# Create the customers_table DataFrame
# CREATE TABLE customers_table (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );
customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    4919528|             5|
|   51451778|             3|
|   12713799|             1|
|    8673341|             1|
|   42146698|             2|
+-----------+--------------+
only showing top 5 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
# CREATE TABLE products_table (
#   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#   product_title TEXT
# );
products_df = df.select(['product_id','product_title']).drop_duplicates()
products_df.show(5)

products_df.columns

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00N2BW2PK|Fitbit Charge HR ...|
|B001D8ZGAM|Aquasentials Easy...|
|B002NVSCQI|Beans72 Organic B...|
|B001O0BL9E|Manuka Health - M...|
|B00AYIM9Y8|BulkSupplements P...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# Create the review_id_table DataFrame. 

# CREATE TABLE review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE -- this should be in the formate yyyy-mm-dd
# );


# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R3EQSTM9PWRAL|     650634|B0091LBZSU|     578484426| 2015-08-31|
| RBWPRK17XKIXD|   19827510|B00PWW3LQ6|     456433146| 2015-08-31|
| RRSLOAF273XFC|    1520474|B00DKEWA92|     460764511| 2015-08-31|
|R3S8W9Q6SWIT8O|   23905905|B0015R3A7M|     135102038| 2015-08-31|
|R3QQ6NSLRVBFJC|   28215779|B006B8U8BG|     200666829| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Create the vine_table. DataFrame

# CREATE TABLE vine_table (
#   review_id TEXT PRIMARY KEY,
#   star_rating INTEGER,
#   helpful_votes INTEGER,
#   total_votes INTEGER,
#   vine TEXT,
#   verified_purchase TEXT
# );

vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazonreviews.cqbgcjbaetrj.us-west-1.rds.amazonaws.com:5432/amazon_reviews"
config = {"user":"postgres", 
          "password": "12345678", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_no_dup_df =products_df.drop_duplicates(subset=['product_id'])
products_no_dup_df.show(20)

DataFrame[product_id: string, product_title: string]

In [ ]:
# Write products_df to table in RDS
# about 3 min

products_no_dup_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)